In [ ]:
!pip install langchain langchain-community chromadb pypdf sentence-transformers openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 4.6 MB/s eta 0:0

In [ ]:
import os
from google.colab import userdata
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import Document

# --- Step 3: Set up the API Key and Environment ---
# IMPORTANT: Before running, you must add your OpenRouter API key to Colab's secrets.
# 1. Click the 'Key' icon on the left sidebar.
# 2. Add a new secret named "OPENROUTER_API_KEY" and paste your key as the value.
try:
    os.environ["OPENROUTER_API_KEY"] = userdata.get("OPENROUTER_API_KEY")
    api_key_found = True
except Exception as e:
    print("ERROR: Could not find the OPENROUTER_API_KEY secret.")
    print("Please add your OpenRouter API key to Colab's secrets (on the left sidebar) and try again.")
    api_key_found = False


if api_key_found:
    # --- Step 4: Load and Process the PDF Document ---
    # Updated the pdf_path to the new file you provided.
    pdf_path = "/content/drive/MyDrive/MACHINE LEARNING(R17A0534).pdf"
    if not os.path.exists(pdf_path):
        print(f"ERROR: The file '{pdf_path}' was not found.")
        print("Please upload the 'RIL-IAR-2025.pdf' file to your Colab session.")
    else:
        print("Loading and processing the PDF... this may take a moment.")
        # Load the PDF
        loader = PyPDFLoader(pdf_path)
        pages = loader.load_and_split()

        # Split the document into smaller chunks for processing
        pdf_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len
        )
        docs = pdf_splitter.split_documents(pages)
        documents = [Document(page_content=doc.page_content) for doc in docs]

        # --- Step 5: Create Text Embeddings and Vector Store ---
        # This converts the text chunks into numerical vectors for similarity searching.
        print("Creating text embeddings and vector store...")
        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2",
            model_kwargs={'device': 'cpu'}
        )
        vector_db = Chroma.from_documents(
            documents,
            embedding=embeddings
        )

        # --- Step 6: Set Up the Conversational AI Model ---
        # This configures the chatbot's "brain" and memory.
        print("Setting up the conversational AI...")
        # Set up conversational memory to remember the chat history
        memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )

        # Initialize the Language Model (LLM) through OpenRouter
        llm = ChatOpenAI(
            model="openai/gpt-3.5-turbo",
            temperature=0.2,
            openai_api_base="https://openrouter.ai/api/v1",
            max_tokens=500,
            openai_api_key=os.environ["OPENROUTER_API_KEY"]
        )

        # Combine the retriever (from the vector store) and the LLM into a conversational chain
        qa_chain = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=vector_db.as_retriever(),
            memory=memory
        )

        print("\n✅ Setup complete! The chatbot is ready.")
        print("You can now ask questions about the Reliance Industries 2024-25 Annual Report.")
        print("Type 'Exit' to end the chat.")
        print("-" * 50)

        # --- Step 7: Start the Real-time Interaction Loop ---
        while True:
            try:
                question = input("User: ")
                if question.lower().strip() == "exit":
                    print("Bot: Thank you for chatting.If you have any queries, you can ask")
                    break
                if not question.strip():
                    continue

                # Get the answer from the QA chain
                answer = qa_chain({"question": question})
                print("Bot:", answer["answer"])

            except Exception as e:
                print(f"An error occurred: {e}")
                break

Loading and processing the PDF... this may take a moment.
Creating text embeddings and vector store...


/tmp/ipython-input-3680790632.py:50: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

Setting up the conversational AI...


/tmp/ipython-input-3680790632.py:63: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(
/tmp/ipython-input-3680790632.py:69: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(



✅ Setup complete! The chatbot is ready.
You can now ask questions about the Reliance Industries 2024-25 Annual Report.
Type 'Exit' to end the chat.
--------------------------------------------------
User: what is machine learning?


/tmp/ipython-input-3680790632.py:100: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain({"question": question})


Bot: Machine learning is the process of programming computers to optimize a performance criterion using example data or past experience. It involves having a model defined with parameters that need to be optimized using training data or past experiences. The model can be predictive to make future predictions or descriptive to gain knowledge from data, or both. Arthur Samuel coined the term "Machine Learning" in 1959, defining it as giving computers the ability to learn without being explicitly programmed. There is no universally accepted definition for machine learning, and different authors define it differently.
User: What are the components of a learning process? Explain each component in detail.
Bot: The components of a learning process, whether for humans or machines, are data storage, abstraction, generalization, and evaluation.
User: Explain supervised, unsupervised, and reinforcement learning with examples.
Bot: Sure! Here are explanations of supervised, unsupervised, and reinf

In [ ]:
!pip install pypdf
